Build & Freeze Feature Datasets
🎯 Objetivo

Este notebook é responsável por gerar e congelar (freeze) os datasets finais de features utilizados em modelagem e simulações do projeto BRMP.
A intenção é garantir reprodutibilidade, consistência e comparabilidade entre diferentes modelos (LogReg, XGBoost, LightGBM, CatBoost), evitando recalcular rolling/expanding a cada experimento.

Regra principal: a engenharia de variáveis deve ser calculada uma vez e persistida em data/processed/.

✅ Entradas (Inputs)

Os dados de entrada já passaram por padronização/limpeza e estão em formato .parquet em data/interim/.

data/interim/fd_2012_2023.parquet
Base histórica para treino (Football-Data, Série A)

data/interim/fd_2024.parquet
Temporada usada para calibração (out-of-sample)

data/interim/fd_2025.parquet
Temporada usada para teste final (out-of-sample)

📌 Observação: Odds existem na fonte, mas não entram aqui.
Este notebook gera um feature set “puro” (sem odds), que servirá como baseline e comparação.

📤 Saídas (Outputs)

Os datasets finais são salvos em data/processed/:

data/processed/features_train_fd_2012_2023.parquet

data/processed/features_cal_fd_2024.parquet

data/processed/features_test_fd_2025.parquet

Cada arquivo contém:

metadados mínimos (data, times, ano)

target (resultado: H, D, A)

features numéricas congeladas (rolling, strength, gaps, etc.)

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("src exists?", (PROJECT_ROOT / "src").exists())








PROJECT_ROOT: /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction
src exists? True


In [7]:
from src.data.features import build_features, FeatureConfig

print("Import OK ✅")


Import OK ✅


In [8]:
INTERIM = PROJECT / "data/interim"
PROCESSED = PROJECT / "data/processed"

PROCESSED.mkdir(parents=True, exist_ok=True)

In [9]:
fd_train_base = pd.read_parquet(INTERIM / "fd_2012_2023.parquet")
fd_cal_base   = pd.read_parquet(INTERIM / "fd_2024.parquet")
fd_test_base  = pd.read_parquet(INTERIM / "fd_2025.parquet")


In [11]:
FEATURE_CONFIG = FeatureConfig(
    rolling_n=5,
    min_periods=1,
    use_season_baseline=True
)


In [15]:
feat_train = build_features(
    df=fd_train_base.copy(),
    cfg=FEATURE_CONFIG
)




In [16]:
feat_cal = build_features(
    df=fd_cal_base.copy(),
    cfg=FEATURE_CONFIG
)


In [17]:
feat_test = build_features(
    df=fd_test_base.copy(),
    cfg=FEATURE_CONFIG
)


In [20]:
META = ["data","ano_campeonato","time_mandante","time_visitante"]
TARGET = "resultado"

FEATURES = [  # a lista final que você já validou
    'home_attack_roll','home_defense_roll',
    'away_attack_roll','away_defense_roll',
    'attack_gap','defense_gap','pts_gap_roll',
    'home_adv_global','home_winrate_global',
    'home_gf_roll','home_ga_roll',
    'away_gf_roll','away_ga_roll',
    'home_gf_roll_home','home_ga_roll_home',
    'away_gf_roll_away','away_ga_roll_away',
    'home_pts_expanding','away_pts_expanding',
    'league_total_goals_mean',
    'home_goal_diff','home_win'
]


In [21]:
import numpy as np

for df_ in (feat_train, feat_cal, feat_test):
    df_[FEATURES] = (
        df_[FEATURES]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(0.0)
    )


In [22]:
assert feat_train[FEATURES].isna().sum().sum() == 0
assert feat_cal[FEATURES].isna().sum().sum() == 0
assert feat_test[FEATURES].isna().sum().sum() == 0

print("✅ Features limpas (sem NA)")


✅ Features limpas (sem NA)


In [23]:
PROCESSED = PROJECT / "data/processed"

feat_train.to_parquet(PROCESSED / "features_train_fd_2012_2023.parquet", index=False)
feat_cal.to_parquet(PROCESSED / "features_cal_fd_2024.parquet", index=False)
feat_test.to_parquet(PROCESSED / "features_test_fd_2025.parquet", index=False)

print("✅ Feature datasets congelados com sucesso")


✅ Feature datasets congelados com sucesso


In [24]:
%cd /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction


/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction


In [27]:
!git rm -r --cached data || true


fatal: pathspec 'data' did not match any files


In [28]:
!git rm -r --cached models || true


fatal: pathspec 'models' did not match any files


In [29]:
!git rm --cached *.parquet 2>/dev/null || true
!git rm --cached *.csv 2>/dev/null || true
!git rm --cached *.pkl 2>/dev/null || true


In [30]:
!git status


Refresh index: 100% (23/23), done.
On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore
	modified:   notebooks/01_eda_brasileirao.ipynb
	modified:   notebooks/02_compare_sources.ipynb
	modified:   src/data/features.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/01_eda_brasileirao_v0_rascunho.ipynb
	notebooks/03_build_feature_datasets.ipynb
	notebooks/04_train_and_calibrate_logreg.ipynb
	reports/
	src/__init__.py
	src/data/__init__.py
	src/utils/__init__.py

no changes added to commit (use "git add" and/or "git commit -a")


In [31]:
!git status --ignored


On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore
	modified:   notebooks/01_eda_brasileirao.ipynb
	modified:   notebooks/02_compare_sources.ipynb
	modified:   src/data/features.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/01_eda_brasileirao_v0_rascunho.ipynb
	notebooks/03_build_feature_datasets.ipynb
	notebooks/04_train_and_calibrate_logreg.ipynb
	reports/
	src/__init__.py
	src/data/__init__.py
	src/utils/__init__.py

Ignored files:
  (use "git add -f <file>..." to include in what will be committed)
	data/
	models/
	src/__pycache__/
	src/data/__pycache__/
	src/models/__init__.py
	src/utils/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


In [32]:
!git ls-files


.gitignore
README.md
dashboards/streamlit_app.py
main.py
notebooks/01_eda_brasileirao.ipynb
notebooks/02_compare_sources.ipynb
notebooks/prepare_football_data_2025.ipynb
requirements.txt
src/config/model_params.yaml
src/config/paths.yaml
src/data/clean.py
src/data/features.py
src/data/ingest.py
src/models/evaluate.py
src/models/predict.py
src/models/train.py
src/utils/helpers.py
src/utils/metrics.py
src/utils/team_aliases.py
src/utils/validation.py
tests/test_data_integrity.py
tests/test_features.py
tests/test_model_output.py


In [33]:
!echo "notebooks/01_eda_brasileirao_v0_rascunho.ipynb" >> .gitignore
